In [1]:
import pandas as pd
import numpy as np
import datetime 
import re

In [2]:
# fr.head(20)

In [3]:
res=[[]]
gauge_list = []
nm_list = []
dicts = {}

fr = pd.read_excel('Кама_данные.xlsx')
ind=fr.loc[fr.iloc[:,0] == "Код поста"].index
fy=int(fr.iloc[ind[0]+1,1])#первый год
for j in ind:#итерация по отдельным таблицам
    nm=[fr.iloc[j,1]]#код поста
    nm_list.append(nm)
    y=int(fr.iloc[j+1,1])#год
    gauge = [fr.iloc[j+2,1]]
    gauge_list.append(gauge)
    
    fd=datetime.datetime(y, 1, 1)#начало года
    ld=datetime.datetime(y+1, 1, 1)#конец года, не включая
    listOfDates=[date for date in np.arange(fd,ld,datetime.timedelta(days=1))]#лист с датами в этом году
    yd=fr.iloc[j+5:j+36,1:13]
    if y%4!=0:#не весокосный год, т.е. остаток от делления на 4 не равен 0:
        #ravel делает из 2d 1d строка за строкой, при этом где-то будут пустые ячейки т.к. не все месяцы по 31 день 
        d=np.delete(yd.values.transpose().ravel(),[59,60,61,123,185,278,340])
    else:
        d=np.delete(yd.values.transpose().ravel(),[60,61,123,185,278,340])
    r=pd.DataFrame(d, columns=nm, index=listOfDates)#делаем datarfame, который по сути series
    if fy==y:
        res[-1].append(r)
    #если пошел следующий год, лист с датафреймами из старого кода соеденяется, а новый датафрейм помещается в лист и 
    #становится основой для сбора расходов со следующего года
    else:
        res[-1]=pd.concat(res[-1], axis=1,sort=False)
        res.append([r])
        fy=y
#последний год объеденяется в единый фрейм уже вне тела цикла
res[-1]=pd.concat(res[-1], axis=1,sort=False)
#наконец все в месте соеденяется
res=pd.concat(res, axis=0,sort=False)

In [4]:
res=res.astype(str)#разные типы данных нам не к чему

In [5]:
#нб - не было в 0, а те ячейки где были дополнительные символы разделяем по пробелу и берем первую часть - число
def str_int(x):
    if x=="нб":
        return 0
    else:
        return float(x.replace(",",".").split(" ")[0])

In [6]:
res=res.applymap(str_int)# применяем функцию ко всем элементам фрейма

In [7]:
res

,76007,76089,76190,76284,76364,76486,76574,76597
2015-01-01,74.1,172.0,59.9,42.1,64.7,18.7,423.0,24.3
2015-01-02,74.1,171.0,62.2,41.6,63.8,18.5,420.0,23.5
2015-01-03,74.1,170.0,54.3,42.1,63.8,18.5,416.0,23.9
2015-01-04,74.1,165.0,53.4,38.9,63.8,18.3,414.0,23.4
2015-01-05,74.1,165.0,54.7,38.9,62.9,18.5,412.0,23.3
...,...,...,...,...,...,...,...,...
2019-12-27,175.0,266.0,78.0,38.6,51.5,18.8,849.0,41.1
2019-12-28,173.0,263.0,77.5,43.4,53.8,18.8,849.0,42.3
2019-12-29,171.0,261.0,73.5,45.1,51.5,18.5,829.0,42.7
2019-12-30,170.0,257.0,70.5,48.5,51.0,18.5,809.0,42.7


In [8]:
df = res.copy()
df['date'] = df.index

In [9]:
df

,76007,76089,76190,76284,76364,76486,76574,76597,date
2015-01-01,74.1,172.0,59.9,42.1,64.7,18.7,423.0,24.3,2015-01-01
2015-01-02,74.1,171.0,62.2,41.6,63.8,18.5,420.0,23.5,2015-01-02
2015-01-03,74.1,170.0,54.3,42.1,63.8,18.5,416.0,23.9,2015-01-03
2015-01-04,74.1,165.0,53.4,38.9,63.8,18.3,414.0,23.4,2015-01-04
2015-01-05,74.1,165.0,54.7,38.9,62.9,18.5,412.0,23.3,2015-01-05
...,...,...,...,...,...,...,...,...,...
2019-12-27,175.0,266.0,78.0,38.6,51.5,18.8,849.0,41.1,2019-12-27
2019-12-28,173.0,263.0,77.5,43.4,53.8,18.8,849.0,42.3,2019-12-28
2019-12-29,171.0,261.0,73.5,45.1,51.5,18.5,829.0,42.7,2019-12-29
2019-12-30,170.0,257.0,70.5,48.5,51.0,18.5,809.0,42.7,2019-12-30


In [10]:
df.reset_index(drop=True, inplace=True)
df['date'] = pd.to_datetime(df['date']).apply(lambda x: datetime.datetime.strftime(x, '%#d %#m %Y'))
# df.columns = ['Q', 'date']
df=df.astype(str)
df

,76007,76089,76190,76284,76364,76486,76574,76597,date
0,74.1,172.0,59.9,42.1,64.7,18.7,423.0,24.3,1 1 2015
1,74.1,171.0,62.2,41.6,63.8,18.5,420.0,23.5,2 1 2015
2,74.1,170.0,54.3,42.1,63.8,18.5,416.0,23.9,3 1 2015
3,74.1,165.0,53.4,38.9,63.8,18.3,414.0,23.4,4 1 2015
4,74.1,165.0,54.7,38.9,62.9,18.5,412.0,23.3,5 1 2015
...,...,...,...,...,...,...,...,...,...
1821,175.0,266.0,78.0,38.6,51.5,18.8,849.0,41.1,27 12 2019
1822,173.0,263.0,77.5,43.4,53.8,18.8,849.0,42.3,28 12 2019
1823,171.0,261.0,73.5,45.1,51.5,18.5,829.0,42.7,29 12 2019
1824,170.0,257.0,70.5,48.5,51.0,18.5,809.0,42.7,30 12 2019


In [11]:
df2 = pd.DataFrame()
for i in df.columns:
    df2[i] = df["date"] + ' ' + df[i]
df2 = df2.drop(columns=['date'])

In [12]:
df2

,76007,76089,76190,76284,76364,76486,76574,76597
0,1 1 2015 74.1,1 1 2015 172.0,1 1 2015 59.9,1 1 2015 42.1,1 1 2015 64.7,1 1 2015 18.7,1 1 2015 423.0,1 1 2015 24.3
1,2 1 2015 74.1,2 1 2015 171.0,2 1 2015 62.2,2 1 2015 41.6,2 1 2015 63.8,2 1 2015 18.5,2 1 2015 420.0,2 1 2015 23.5
2,3 1 2015 74.1,3 1 2015 170.0,3 1 2015 54.3,3 1 2015 42.1,3 1 2015 63.8,3 1 2015 18.5,3 1 2015 416.0,3 1 2015 23.9
3,4 1 2015 74.1,4 1 2015 165.0,4 1 2015 53.4,4 1 2015 38.9,4 1 2015 63.8,4 1 2015 18.3,4 1 2015 414.0,4 1 2015 23.4
4,5 1 2015 74.1,5 1 2015 165.0,5 1 2015 54.7,5 1 2015 38.9,5 1 2015 62.9,5 1 2015 18.5,5 1 2015 412.0,5 1 2015 23.3
...,...,...,...,...,...,...,...,...
1821,27 12 2019 175.0,27 12 2019 266.0,27 12 2019 78.0,27 12 2019 38.6,27 12 2019 51.5,27 12 2019 18.8,27 12 2019 849.0,27 12 2019 41.1
1822,28 12 2019 173.0,28 12 2019 263.0,28 12 2019 77.5,28 12 2019 43.4,28 12 2019 53.8,28 12 2019 18.8,28 12 2019 849.0,28 12 2019 42.3
1823,29 12 2019 171.0,29 12 2019 261.0,29 12 2019 73.5,29 12 2019 45.1,29 12 2019 51.5,29 12 2019 18.5,29 12 2019 829.0,29 12 2019 42.7
1824,30 12 2019 170.0,30 12 2019 257.0,30 12 2019 70.5,30 12 2019 48.5,30 12 2019 51.0,30 12 2019 18.5,30 12 2019 809.0,30 12 2019 42.7


In [13]:
df2.to_excel('Кама_для_продления.xlsx')

In [14]:
flat_gauge = [item for sublist in gauge_list for item in sublist]
flat_nm = [item for sublist in nm_list for item in sublist]


In [15]:
list(set(flat_gauge))

['УФА - р.УФА - г. КРАСНОУФИМСК',
 'КАМА - р. КАМА - пгт ГАЙНЫ',
 'Чепца - P.ЧEПЦA - Г.ГЛAЗOB',
 'Вятка - P.BЯTKA - Г.BЯTCKИE ПOЛЯHЫ',
 'ВИШЕРА - р. ВИШЕРА - пос. РЯБИНИНО',
 'ЧУСОВАЯ - р. ЧУСОВАЯ - пгт ЛЯМИНО',
 'БЕЛАЯ - р.Белая - г.Стерлитамак',
 'ДЕМА - р.Дёма - д.Бочкарёва']

In [16]:
list(set(flat_nm))

[76486, 76007, 76364, 76574, 76597, 76089, 76284, 76190]